In [1]:
!pip install pulp



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 24.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
import time
import json
import pandas as pd
from itertools import product
import os
import ast


# Parameters to generate examples
cardinalities = [100]
dimensions = [30]
distance_types = [0,1]
n_div = 5
left = 0
right = 5

type_atmg = 1
#step = 10

# File paths for storing results
json_filename = "test_logsATMGINT_BF.json"
csv_filename = "test_resultsATMGINT_BF.csv"
json_filename2 = "test_logs2ATMGINT_BF.json"

# Generate list of pairs of cardinalities and dimensions
pairs = product(cardinalities, dimensions)

def save_log_to_json(key, log_data, file_path):
    #Saves log and model to a JSON file with a unique identifier, along with the key

    # Convert the data to a serializable format
    log_data_serializable = {str(k): v for k, v in log_data.items()}

    # Generate a unique identifier for the log
    log_id = hash(json.dumps(log_data_serializable, sort_keys=True))

    # Structure for storing the log with the key
    log_entry = {
        "Key": key,
        "Log Data": log_data_serializable
    }

    # Create or update the JSON file
    if not os.path.exists(file_path):
        with open(file_path, 'w') as f:
            json.dump({}, f, indent=4)

    # Load existing data
    with open(file_path, 'r') as f:
        existing_data = json.load(f)

    # Add the new log with the unique identifier
    existing_data[str(log_id)] = log_entry

    # Write back the updated data
    with open(file_path, 'w') as f:
        json.dump(existing_data, f, indent=4)

    return log_id  # Return the unique identifier to reference this log

# Function to append to a CSV file
def append_results_to_csv(results_dict, file_path):
    #Appends data (result) to a CSV file.
    df = pd.DataFrame([results_dict])
    file_exists = os.path.exists(file_path)

    # If the file does not exist or is empty, write headers
    if not file_exists or os.path.getsize(file_path) == 0:
        df.to_csv(file_path, mode='w', index=False, header=True, sep=';')
    else:
        df.to_csv(file_path, mode='a', index=False, header=False, sep=';')  # Append without headers


def retrieve_example(cardinality, dimension):
  #retrieves the log and model from the JSON file given the cardinality and dimension of the problem
  #it retrieves it only for distance type 0 (stamp only), to retrieve the "original" log

  filepath = "/content/drive/MyDrive/lp_solver_logs.json"

  # Load JSON data from a file
  with open(filepath, "r") as f:
      json_data = json.load(f)

  key_pattern = f"{cardinality}-{dimension}-0"

  # Find the entry with the corresponding key
  for key, entry in json_data.items():
      if entry.get("Key") == key_pattern:
          # Get the Model and Log from the "Log Data"
          model = ast.literal_eval(entry.get("Log Data", {}).get("Model"))
          log = entry.get("Log Data", {}).get("Log")
          return model, log

  # If the key pattern is not found, return a message
  return {"error": "No data found for the given key pattern"}


In [ ]:
from STPN import STPN

# Dictionary to store results
lp_results = {}

# LP-based experiments loop
for cardinality, dimension in pairs:

    # Generate\retrieve example
    #model, log, = retrieve_example(cardinality, dimension)
    model,log = STPN.random_example(dimension,left,right, cardinality, random_intervals=True)
    print(model)
    print(log)

    #print(model)

    for distance_type in distance_types:
        key = f"{cardinality}-{dimension}-{distance_type}"
        print(f"Cardinality: {cardinality}, Dimension: {dimension}, Distance Type: {distance_type}")

        # Save the log data to the JSON file and get the unique ID
        #The log is initially saved to the "backup" json file in order to have it saved even if the execution is interrupted
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename2)

        start_time = time.time()
        max_dist_LP, opt_point_LP, numVars, numConstr = model.solver_lp(log, distance_type)
        elapsed_time_LP = time.time() - start_time
        """
        start_time_BF = time.time()
        max_dist_BF, opt_point_BF = model.solver_bf(log, distance_type,n_div)
        elapsed_time_BF = time.time() - start_time_BF
        """
        # Save the log and model in json file
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename)

        # Store the results
        result_data = {
            'Key': key,
            'Dimension': dimension,
            'Cardinality': cardinality,
            'Distance Type': distance_type,
            'Left': left,
            'Right': right,
            #'Model': str(model),  # Ensure this is serializable
            #'Log': str(L),  # Ensure this is serializable
            'Elapsed Time (LP)': elapsed_time_LP,
            #'Elapsed Time (BF)': elapsed_time_BF,
            #'Number of Variables (LP)': numVars,
            #'Number of Constraints (LP)': numConstr,
            'Solution (LP)': str(opt_point_LP),  # Ensure this is serializable
            #'Solution (BF)': str(opt_point_BF),  # Ensure this is serializable
            'Max Distance (LP)': max_dist_LP,
            #'Max Distance (BF)': max_dist_BF,
        }

        append_results_to_csv(result_data, csv_filename)


        # Display the results
        print(f"Elapsed Time (LP): {elapsed_time_LP:.6f} seconds")
        #print(f"Elapsed Time (BF): {elapsed_time_BF:.6f} seconds")
        #print(f"Number of Variables (LP): {numVars}")
        #print(f"Number of Constraints (LP): {numConstr}")
        print(f"Max Distance (LP): {max_dist_LP}")
        #print(f"Max Distance (BF): {max_dist_BF}")

print(f"LP-based results successfully appended {csv_filename}.")



STPN contraints: [(0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000), (0, 1000)]
[[547.2, 1265.33, 1417.67, 1665.19, 2115.63, 3074.24, 3221.34, 4019.7, 4285.7, 4930.59, 5378.65, 5407.15, 5795.99, 6266.27, 6693.2], [634.14, 874.11, 1294.43, 1663.12, 2264.82, 2824.66, 3427.53, 3856.21, 3972.91, 4746.19, 5163.51, 5599.67, 6503.0, 7001.67, 7233.22], [425.64, 904.61, 970.85, 1150.75, 2137.0, 2981.38, 3930.16, 4872.54, 5213.55, 5232.7, 5649.86, 6372.55, 6496.65, 7154.58, 7200.76], [756.64, 1625.76, 2490.23, 2765.45, 3419.45, 4278.32, 4781.67, 5549.95, 6209.89, 6390.47, 7388.51, 7718.52, 8016.63, 8082.76, 8545.8], [157.17, 961.26, 1723.19, 2025.04, 2449.51, 2936.8, 3785.65, 4649.09, 5153.94, 5879.58, 6363.83, 6747.89, 7481.4, 7744.57, 8145.3], [791.59, 1692.8, 2355.17, 2356.0, 2702.67, 3347.82, 4187.43, 4942.95, 5277.48, 6188.58, 6436.57, 6998.47, 7552.4, 8502.62, 8872.74], [938.76, 1424.28, 

In [8]:
from ATMG import ATMG

# Dictionary to store results
lp_results = {}

# LP-based experiments loop
for cardinality, dimension in pairs:

    # Generate\retrieve example
    #model, log, = retrieve_example(cardinality, dimension)

    # Generate random example
    model, log = ATMG.random_example(dimension,left,right, cardinality, type_atmg, random_intervals=True)
    print(model.max_point())
    print(model)
    print(log)
    #print(model)


    for distance_type in distance_types:
        key = f"{cardinality}-{dimension}-{distance_type}"
        print(f"Cardinality: {cardinality}, Dimension: {dimension}, Distance Type: {distance_type}")

        # Save the log data to the JSON file and get the unique ID
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename2)
        #REMOVE LOG AND MODEL FROM JSON IF SOLUTION NOT ENDED

        start_time = time.time()
        max_dist_LP, opt_point_LP, numVars, numConstr = model.LPSolver(log.copy(), distance_type)
        elapsed_time_LP = time.time() - start_time

        """
        start_time_BF = time.time()
        max_dist_BF, opt_point_BF = model.solver_bf(log, distance_type,n_div)
        elapsed_time_BF = time.time() - start_time_BF
        """
        # Save the log data to the JSON file and get the unique ID
        log_id = save_log_to_json(key, {"Model": str(model), "Log": log}, json_filename)
        #REMOVE LOG AND MODEL FROM JSON IF SOLUTION NOT ENDED


        # Store the results
        result_data = {
            'Key': key,
            'Dimension': dimension,
            'Cardinality': cardinality,
            'Distance Type': distance_type,
            'Left': left,
            'Right': right,
            #'Model': str(model),  # Ensure this is serializable
            #'Log': str(L),  # Ensure this is serializable
            'Elapsed Time (LP)': elapsed_time_LP,
            #'Elapsed Time (BF)': elapsed_time_BF,
            #'Number of Variables (LP)': numVars,
            #'Number of Constraints (LP)': numConstr,
            'Solution (LP)': str(opt_point_LP),  # Ensure this is serializable
            #'Solution (BF)': str(opt_point_BF),  # Ensure this is serializable
            'Max Distance (LP)': max_dist_LP,
            #'Max Distance (BF)': max_dist_BF,
        }

        append_results_to_csv(result_data, csv_filename)


        # Display the results
        print(f"Elapsed Time (LP): {elapsed_time_LP:.6f} seconds")
        #print(f"Elapsed Time (BF): {elapsed_time_BF:.6f} seconds")
        print(f"Max Distance (LP): {max_dist_LP}")
        #print(f"Max Distance (BF): {max_dist_BF}")

        print()

        # Save the results to both JSON and CSV files
        #append_results_to_json(lp_results, json_filename)

print(f"Results successfully appended {csv_filename}.")

{'t0': 't0', 't28': 't1', 't27': 't2', 't26': 't3', 't25': 't4', 't24': 't5', 't23': 't6', 't22': 't7', 't21': 't8', 't20': 't9', 't19': 't10', 't18': 't11', 't17': 't12', 't16': 't13', 't15': 't14', 't14': 't15', 't13': 't16', 't12': 't17', 't11': 't18', 't10': 't19', 't9': 't20', 't8': 't21', 't7': 't22', 't6': 't23', 't5': 't24', 't4': 't25', 't3': 't26', 't2': 't27', 't1': 't28', 't29': 't29'}
[4.54, 7.63, 8.46, 11.42, 11.780000000000001, 9.1, 13.77, 9.42, 9.59, 10.6, 10.17, 10.93, 14.010000000000002, 9.24, 9.42, 12.33, 11.08, 12.07, 12.68, 10.95, 10.66, 7.62, 11.14, 12.17, 7.720000000000001, 8.11, 7.3, 10.14, 9.74, 20.470000000000002]
Places: {'p29', 'p25', 'p40', 'p39', 'p37', 'p47', 'p52', 'p27', 'p30', 'p0', 'p22', 'p11', 'p13', 'p21', 'p35', 'p18', 'p9', 'p4', 'p24', 'p6', 'p15', 'p46', 'p20', 'p41', 'p8', 'p23', 'p3', 'p53', 'p38', 'p42', 'p19', 'p5', 'p2', 'p17', 'p26', 'p56', 'p44', 'p55', 'p32', 'p48', 'p16', 'p54', 'p49', 'p28', 'p31', 'p51', 'p34', 'p36', 'p10', 'p50', '